In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import requests

import pandas as pd

client_id = "45da58455f7f46d5b30b0b8a117fa452"
client_secret = "7dd2efbeaecd4fd08136e682f7610f48"


# 獲取 access_token
def get_access_token():
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials"
    }
    response = requests.post(url, headers=headers, data=data, auth=(client_id, client_secret))
    return response.json().get('access_token')


# 查詢歌手專輯 ID
def get_artist_album_id(artist_id, access_token):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/albums"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    response = requests.get(url, headers=headers)
    data = response.json()

    # 取得專輯 ID
    albums = data.get('items', [])
    artist_album_id = [album["id"] for album in albums]

    return artist_album_id


# 主程式
def main():
    access_token = get_access_token()

    # 讀取 CSV
    artist_ids_path = r"C:\Users\T14 Gen 3\Desktop\KKbox_Spotify\artist_ids.csv"
    df = pd.read_csv(artist_ids_path)
      
    artist_album_ids = {}

    for index, row in df.iterrows():
        artist_id = row['artist_id']
        album_ids = get_artist_album_id(artist_id, access_token)
        artist_album_ids[artist_id] = album_ids
   

    # 將查詢結果存入 DataFrame
    result_data = []
    for artist_id, album_ids in artist_album_ids.items():
        for album_id in album_ids:  # 將每個專輯 ID 展開
            result_data.append({'Artist_id': artist_id, 'Artist_album_id': album_id})
    result_df = pd.DataFrame(result_data)

    # 保存結果到 CSV
    result_csv_path = r"C:\Users\T14 Gen 3\Desktop\KKbox_Spotify\artist_album_ids.csv"
    result_df.to_csv(result_csv_path, index=False)
    

if __name__ == "__main__":
    main()